In [ ]:
import pandas as pd
import numpy as np
from requests import get
from dataclasses import asdict
import xmltodict

from my_scientific_profile.database.papers import load_all_papers_from_s3
from my_scientific_profile.web_app.extensions import s3_client, S3_BUCKET

In [ ]:
papers = load_all_papers_from_s3(s3_client=s3_client, s3_bucket=S3_BUCKET)

In [ ]:
df = pd.json_normalize(asdict(p) for p in papers)
df.head()

In [ ]:
df[["doi", "title", "abstract"]].info()

In [ ]:
df.loc[df.abstract.isna()][["doi", "title", "abstract", "journal.url"]]

In [ ]:
def retrieve_doi(doi: str):
    header = {
        "Accept": "application/json",
        "X-ELS-APIKey": "a68adb8cc0d4dd669baac77323b0bcb9",
        # "X-ELS-Insttoken": "",
        "view": "META",
    }
    return xmltodict.parse(
        get(url=f"https://api.elsevier.com/content/abstract/doi/{doi}", params=header).text
    )

In [ ]:
retrieve_doi("10.1038/s41586-022-04501-x")

In [ ]:
def retrieve_orcid(orcid: str):
    header = {
        "Accept": "application/json",
        "apiKey": "a68adb8cc0d4dd669baac77323b0bcb9",
    }
    return xmltodict.parse(
        get(url=f"https://api.elsevier.com/content/author/orcid/{orcid}", params=header).text
    )

In [ ]:
retrieve_orcid("0000-0001-9945-1271")

## Open access PDF

In [ ]:
df.loc[df["open_access.pdf_url"].isna()].loc[df.abstract.isna()]